# EEG analysis in MNE Python


### General quality checks of EEG data and preprocssing
### Aim: Clean EEG data

author: Carina Forster

contact: forster@cbs.mpg.de

last updated: 14.10.2024

Historically, MNE was a software for computing cortically constrained Minimum Norm Estimates from MEG and EEG data. The historical core functions of MNE were written by Matti Hämäläinen in Boston and originate in part from the Elekta software that is shipped with its MEG systems. The FIFF is Elektas Functional Imaging File Format that goes along with .fif file extensions and is natively used by its MEG systems. For these reasons the MNE software is internally relying on the FIFF files. 

Today the situation is a bit different though. MNE is nowadays developed mostly in Python by an international team of researchers from diverse laboratories and has widened its scope.

MNE supports advanced sensor space analyses for EEG, temporal ICA, many different file formats and many other inverse solvers, for example beamformers. 

Some of our contributors even use it for intracranial data. If you want, MNE can be thought of as MEG’n’EEG.

If you haven't installed MNE yet, please follow the [instructions](https://mne.tools/stable/install/index.html) on the homepage (install with conda is recommended)
Update: Due to license issues with ananconda I recommend using [miniforge](https://github.com/conda-forge/miniforge) to install conda (and mamba)

Here some legal stuff on [anaconda licensing](https://www.anaconda.com/blog/update-on-anacondas-terms-of-service-for-academia-and-research)

we start by importing important packages

In [1]:
import mne
import numpy as np
import os
from pathlib import Path

In [23]:
# make sure you use the stable version of mne (currently 1.7.1 but check for updates)
mne.__version__

'1.8.0'

In [25]:
# this command makes sure plots are shown in the notebook (they don't open a new window)
#%matplotlib inline
# this command enables matplotlib to use the qt backend that allows you to view data interactively (click and scroll through data)
%matplotlib qt

In [26]:
# get rid of the sometimes excessive logging output
mne.set_log_level('error')

## Loading data

### make sure to change the path to where you stored the data

In [32]:
# let's load some sample data from MNE
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample',
                                    'sample_audvis_raw.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file)

In [33]:
# the data is not loaded into memory yet, only the file header is read
raw._data

AttributeError: 'Raw' object has no attribute '_data'

<div class="alert alert-block alert-warning">
<b>Discussion:</b> Data is not loaded. Why would that be smart?

EEG data is very memory-heavy

</div>

In [34]:
# let's load the data into memory
raw.load_data();

In [35]:
# now the data is loaded and we can access it
raw._data

array([[ 9.64355481e-12,  0.00000000e+00,  0.00000000e+00, ...,
        -1.92871096e-12,  2.89306644e-12,  3.85742192e-12],
       [-4.82177740e-12, -2.89306644e-12, -9.64355481e-13, ...,
        -9.64355481e-13, -9.64355481e-13, -1.92871096e-12],
       [ 1.01074222e-13,  6.31713890e-14,  7.58056668e-14, ...,
        -4.80102556e-13, -6.06445334e-13, -5.93811056e-13],
       ...,
       [ 3.88542173e-05,  4.07510373e-05,  4.09957883e-05, ...,
         6.72453304e-05,  6.68782039e-05,  6.91421504e-05],
       [ 6.58391126e-05,  6.80025648e-05,  6.81779798e-05, ...,
         8.51932390e-05,  8.58948991e-05,  8.89938982e-05],
       [ 2.85661012e-04,  2.83699953e-04,  2.80431520e-04, ...,
         2.64089357e-04,  2.62781984e-04,  2.57552492e-04]])

In [36]:
# View dataset info
raw.info;

<Info | 21 non-empty values
 acq_pars: ACQch001 110113 ACQch002 110112 ACQch003 110111 ACQch004 110122 ...
 bads: 2 items (MEG 2443, EEG 053)
 ch_names: MEG 0113, MEG 0112, MEG 0111, MEG 0122, MEG 0123, MEG 0121, MEG ...
 chs: 204 Gradiometers, 102 Magnetometers, 9 Stimulus, 60 EEG, 1 EOG
 custom_ref_applied: False
 description: acquisition (megacq) VectorView system at NMR-MGH
 dev_head_t: MEG device -> head transform
 dig: 146 items (3 Cardinal, 4 HPI, 61 EEG, 78 Extra)
 events: 1 item (list)
 experimenter: MEG
 file_id: 4 items (dict)
 highpass: 0.1 Hz
 hpi_meas: 1 item (list)
 hpi_results: 1 item (list)
 lowpass: 172.2 Hz
 meas_date: 2002-12-03 19:01:10 UTC
 meas_id: 4 items (dict)
 nchan: 376
 proj_id: 1 item (ndarray)
 proj_name: test
 projs: PCA-v1: off, PCA-v2: off, PCA-v3: off
 sfreq: 600.6 Hz
>

In [37]:
# Select only EEG and EOG channels as well as the stimulus channel
raw_eeg = raw.copy().pick_types(meg = False, eeg = True, eog = True, stim = True)

## Raw signal
We can now plot and look at the raw signal. We can already identify some artifacts in there - we will deal with them at the next step.

In [38]:
# let's plot the data (if you didn't enable the qt backend you can't scroll through the data interactively)
raw_eeg.plot(duration=5, n_channels=10);

Let's check the actual data structure

In [39]:
raw_eeg._data.shape

(69, 166800)

<div class="alert alert-block alert-warning">
<b>Discussion:</b> What are the dimensions representing? 

the first dimension is the number of channels, 

the second dimension the time samples
</div>

In [40]:
# Let's extract the sampling frequency
raw_eeg.info['sfreq']

600.614990234375

<div class="alert alert-block alert-success">
<b>Exercise:</b> Calculate the duration of the EEG recording in minutes and seconds.
</div>

In [41]:
scan_durn = raw_eeg._data.shape[1] / raw_eeg.info['sfreq']
print('Duration of EEG recording = ', scan_durn, 's, or', round(scan_durn / 60, 2), 'min.')

Duration of EEG recording =  277.7153462901591 s, or 4.63 min.


In [42]:
# Let's extract the channel names
raw_eeg.info['ch_names'];

['STI 001',
 'STI 002',
 'STI 003',
 'STI 004',
 'STI 005',
 'STI 006',
 'STI 014',
 'STI 015',
 'STI 016',
 'EEG 001',
 'EEG 002',
 'EEG 003',
 'EEG 004',
 'EEG 005',
 'EEG 006',
 'EEG 007',
 'EEG 008',
 'EEG 009',
 'EEG 010',
 'EEG 011',
 'EEG 012',
 'EEG 013',
 'EEG 014',
 'EEG 015',
 'EEG 016',
 'EEG 017',
 'EEG 018',
 'EEG 019',
 'EEG 020',
 'EEG 021',
 'EEG 022',
 'EEG 023',
 'EEG 024',
 'EEG 025',
 'EEG 026',
 'EEG 027',
 'EEG 028',
 'EEG 029',
 'EEG 030',
 'EEG 031',
 'EEG 032',
 'EEG 033',
 'EEG 034',
 'EEG 035',
 'EEG 036',
 'EEG 037',
 'EEG 038',
 'EEG 039',
 'EEG 040',
 'EEG 041',
 'EEG 042',
 'EEG 043',
 'EEG 044',
 'EEG 045',
 'EEG 046',
 'EEG 047',
 'EEG 048',
 'EEG 049',
 'EEG 050',
 'EEG 051',
 'EEG 052',
 'EEG 054',
 'EEG 055',
 'EEG 056',
 'EEG 057',
 'EEG 058',
 'EEG 059',
 'EEG 060',
 'EOG 061']

In [56]:
# plot the montage and make sure it looks correct
raw_eeg.plot_sensors(show_names=True);

<div class="alert alert-block alert-success">
<b>Exercise:</b>  Plot the data of a visual channel of your choice for 1 second
</div>

In [47]:
# I prefere to use the seaborn package for plotting, so let's import it
import seaborn as sns

data_eeg = raw_eeg.get_data(picks='EEG 057', tmin=3, tmax=4)
print(data_eeg.shape)
# we need to transpose the data to get the right shape for seaborn (samples should be the first dimension)
sns.lineplot(data_eeg.T);

(1, 601)


<div class="alert alert-block alert-info">
<b>Bonus:</b> Can you improve this plot?
</div>


In [49]:
# it's up to you how you want to plot the data, here is an example using matplotlib
import matplotlib.pyplot as plt

plt.plot(np.linspace(3,4,len(data_eeg.T)), data_eeg.T)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (mV)')
plt.title('EEG signal in channel EEG 057')
plt.show()

In [54]:
# You can also plot one single channel using MNE's plotting functions
raw_eeg.plot(picks='EEG 057', duration=1, start=3);

Let's look closer at the channels

In [57]:
# useful method to get an overall summary of the data
raw_eeg.describe()

<Raw | sample_audvis_raw.fif, 69 x 166800 (277.7 s), ~90.8 MB, data loaded>
ch  name     type  unit        min         Q1     median         Q3        max
 0  STI 001  STIM  V          0.00       0.00       0.00       0.00       5.00
 1  STI 002  STIM  V          0.00       0.00       0.00       0.00       5.00
 2  STI 003  STIM  V          0.00       0.00       0.00       0.00       5.00
 3  STI 004  STIM  V          0.00       0.00       0.00       0.00       0.00
 4  STI 005  STIM  V          0.00       0.00       0.00       0.00       0.00
 5  STI 006  STIM  V          0.00       0.00       0.00       0.00       5.00
 6  STI 014  STIM  V          0.00       0.00       0.00       0.00      32.00
 7  STI 015  STIM  V          0.00       0.00       0.00       0.00       0.00
 8  STI 016  STIM  V          0.00       0.00       0.00       0.00       0.00
 9  EEG 001  EEG   µV      -298.79      -3.10       8.18      20.20     299.84
10  EEG 002  EEG   µV      -250.20      38.42      48.1

# Preprocessing

Preprocessing (cleaning, removing artifacts) is essential and not p-hacking.

Remember: Garbage inn, garbage out

Physiological noise: eye movements, heart-beat related artifacts, respiration, \
                     muscle contractions, head movements,
                     drifts (due to excessive sweating)

Non-phyhsiological noise: line noise, power noise

Highly recommended reading: Reproducible EEG Research by [Pernet and colleagues (2020)](https://www.nature.com/articles/s41593-020-00709-0)

### Filtering
Filtering is an important step in EEG analysis - the sensors pick up anything from very slow (<1 Hz) to extremely fast (>200 Hz) oscillations. Not all of these are likely to be related to brain activity. Typically, the data is filtered between 1 and 20-50 Hz to get a relevant range.

<div class="alert alert-block alert-warning">
<b>Discussion:</b>

What is an oscillation? <br /> 

definition based on physics (Cambridge dictionary):
a regular change in strength or direction in a wave or electric current <br />


What is a low-pass or high-pass filter and what is a bandpass filter? <br /> low-pass: allow oscillations lower than a certain frequency to pass <br /> high-pass: allow oscillations higher than a certain frequency to pass <br />
band-pass: allow oscillations between specified frequencies to pass

What is Aliasing? <br /> 
overlapping of frequency components resulting from a sample rate below the Nyquist rate <br />
can create distortions in the signal

</div>

<div class="alert alert-block alert-success">
<b>Exercise:</b>  

Determine the Nquist frequency of the data: <br />  half of the sampling rate: 300 Hz, better 1/3rd: 150 Hz

Was the data online filtered and if so with what frequency: <br /> 
yes, online filter (while recording data) at 172 Hz (anti-aliasing filter) and high-pass filter at 0.1 Hz
</div>

In [58]:
# Let's plot the power spectral density
raw_eeg.compute_psd().plot();

<div class="alert alert-block alert-warning">
<b>Discussion:</b>

What is a PSD?

Power Spectral Density: describes how much power at which frequency

What is 1/f?

Pink Noise: amplitude decrease with increasing frequency (power law)

</div>

In [59]:
# finally let's filter the data (band-pass filter)
raw_eeg_01Hz = raw_eeg.copy().filter(0.1,30)

raw_eeg_01Hz.compute_psd(fmax=40).plot();

precision in the frequency domain is inversely related to precision in the time domain or in other words <br />  =>
sharper frequency cutoff, larger artifacts in the time domain

For a discussion on [filter options](https://mne.tools/stable/auto_tutorials/preprocessing/25_background_filtering.html)

In [61]:
# what other artifacts do we see?
raw_eeg_01Hz.plot();

### Independent component analysis (ICA)

ICA is one of the most common methods used to remove artifacts in the data (such as blinks, heartbeats and so on). 

Briefly, your EEG signal is a mixture of many signals and ICA helps "unmix" or separate them.

We will fit the ICA algorithm, plot the components and then remove those that look like artifacts. 

The details are beyond the scope of the class, but more information can be found on the [MNE homepage](
https://mne.tools/stable/auto_tutorials/preprocessing/40_artifact_correction_ica.html)

In [62]:
# we filter the data with with a 1Hz high-pass filter
# Notes: this is the reason we created a copy of the raw data before filtering
raw_eeg_1Hz = raw_eeg.copy().filter(1, 30);

<div class="alert alert-block alert-warning">
<b>Discussion:</b>

Why do we filter the data with a 1Hz high-pass filter for ICA?

low frequency noise has a high amplitude and would dominate the ICA components (remember the power law)

</div>

In [63]:
# let's plot the PSD of the data
raw_eeg_1Hz.plot_psd(fmax=40);

# do you see the waterfall shape around the filter onset? This is a filter artefact

In [64]:
# let's break the data into smaller chunks (epochs)
tstep = 3.0 # 60 seconds
events_ica = mne.make_fixed_length_events(raw_eeg_1Hz, duration=tstep)
epochs_ica = mne.Epochs(raw_eeg_1Hz, events_ica,
                        tmin=0.0, tmax=tstep,
                        baseline=None,
                        preload=True)
# this is not necessary but it's a good practice to have the data in epochs before running ICA, especially for 
# event related designs

# Try it out on the continuous data

<div class="alert alert-block alert-warning">
<b>Discussion:</b>

ICA on continuous signal or epochs? 

usually, it's a good practice to have the data in epochs before running ICA, especially for 
event-related designs as you don't want to include data you are not interested in (breaks etc.)
also, it is computationally more efficient

What are Pros and cons?

- epochs: faster, noise unrelated to the experiment is removed (e.g. coughing in breaks)
- continuous: slower, no artificial breaks in the data

</div>

In [65]:
# we can plot epochs the same way we can plot continuous data
epochs_ica.plot(n_channels=10, n_epochs=1);

Excurse and not always necessary: [Autoreject package](https://autoreject.github.io/stable/explanation.html) to automatically detect and clean bad epochs

In [69]:
# Excurs: Use automated cleaning pipeline: AutoReject package
from autoreject import AutoReject

ar = AutoReject()

ar.fit(epochs_ica)

reject_log = ar.get_reject_log(epochs_ica)

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=[15, 5])
reject_log.plot('horizontal', ax=ax, aspect='auto')
plt.show()

C:\Users\Carina\miniforge3\envs\mne\Lib\site-packages\autoreject\utils.py:73: UserWarning: 3 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


Running autoreject on ch_type=eeg




Estimated consensus=0.70 and n_interpolate=32


In [70]:
# set up and fit the ICA

# ICA parameters
random_state = 42   # ensures ICA is reproducible each time it's run, # Why 42?
ica_n_components = .99     # Specify n_components as a decimal to set % explained variance, 
# or an integer to set the number of components

# Fit ICA
ica = mne.preprocessing.ICA(n_components=ica_n_components,
                            random_state=random_state,
                            )

# we only fit the ICA to the good epochs
ica.fit(epochs_ica[~reject_log.bad_epochs], decim=3) # what does the tilde symbol do?

Method,fastica
Fit parameters,algorithm=parallelfun=logcoshfun_args=Nonemax_iter=1000
Fit,53 iterations on epochs (47479 samples)
ICA components,25
Available PCA components,56
Channel types,eeg
ICA components marked for exclusion,—


Here are the components identified 

In [71]:
ica.plot_components(inst=epochs_ica);

In [72]:
ica.plot_sources(raw_eeg_1Hz);

if you pass an instance (raw data or epoched data) you can view the ICA components interactively

We can visualize the effects of excluding a particular component from the data.

In [73]:
ica.plot_overlay(raw_eeg_1Hz, exclude=[0], picks='eeg');

<div class="alert alert-block alert-warning">
<b>Discussion:</b>

How do we determine which components to exclude? 

there are many different ways: manual, semi-automatic or fully automated

</div>

In [79]:
# set channel types
raw_eeg.set_channel_types({'EOG 061': 'eog'})

<Raw | sample_audvis_raw.fif, 69 x 166800 (277.7 s), ~90.8 MB, data loaded>

### One way: correlation-based approach

In [80]:
# correlate with EOG channel
ica.exclude = []

eog_indices, eog_scores = ica.find_bads_eog(epochs_ica,
                                            #ch_name=['EOG 061'], # we defined the EOG channels as EOG types earlier
                                            # so not necessary to specify them here
                                            threshold='auto',
                                            measure='zscore'
                                            )

print('Number of EOG components identified: ' + str(len(eog_indices)))

emg_indices, emg_scores = ica.find_bads_muscle(epochs_ica)

print('Number of EMG components identified: ' + str(len(emg_indices)))

# assign the bad EOG components to the ICA.exclude attribute so they can be removed later
ica.exclude = eog_indices + emg_indices

Number of EOG components identified: 1
Number of EMG components identified: 0


In [ ]:
# do the same for ecg 
#ecg_indices, ecg_scores = ica.find_bads_ecg(epochs_ica,
                                            #ch_name=['ECG1', 'ECG2'],
#                                            )
#print('Number of ECG components identified: ' + str(len(ecg_indices)))

#Bonus: Create epochs around Rpeaks, do you see a heart-beat related potential?
# get epochs around Rpeaks
#ecg_epochs = mne.preprocessing.create_ecg_epochs(raw_eeg_1Hz, tmin=-0.5, tmax=0.5)
#ecg_evoked = ecg_epochs.average()
#ecg_evoked.plot_joint();

In [81]:
# plot the correlation scores to see which components are most likely to be EOG
ica.plot_scores(eog_scores);

In [82]:
# useful if you defined n_components as a decimal
explained_var_ratio = ica.get_explained_variance_ratio(epochs_ica)
print(explained_var_ratio)

{'eeg': 0.9928368108601303}


### check out my [preprocessing script](https://github.com/CarinaFo/ExPeCoN_ms/tree/master/code/expecon_ms/eeg/preprocessing) or [IClabel](https://mne.tools/mne-icalabel/dev/index.html) for other options to automate the ICA labelling approach

Nice! We got rid of low and high frequency noise and blinks!

With your own data you might want to spend more time on preprocessing. 
### This is an important step! 

Let's save what we did so far

In [89]:
# define save path
save_dir = Path("C:/", "Users", "Carina", "Desktop", "MNE", "hamburg_2024", "data")

In [92]:
# save the raw data before ICA
raw_eeg_01Hz.save(Path(save_dir, 'sample_before_ica-raw.fif'), overwrite=True) 

In [91]:
# finally apply the ica rejection to the 0.1Hz filtered data
raw_eeg_clean = ica.apply(raw_eeg_01Hz)

In [93]:
# save the clean data after ICA
raw_eeg_clean.save(Path(save_dir, 'sample_after_prepro_and_ica-raw.fif'), overwrite=True)

<div class="alert alert-block alert-success">
<b>Exercise:</b> save the ica solution disk.
</div>

In [94]:
ica.save(Path(save_dir, 'ica.fif'))

Method,fastica
Fit parameters,algorithm=parallelfun=logcoshfun_args=Nonemax_iter=1000
Fit,53 iterations on epochs (47479 samples)
ICA components,25
Available PCA components,56
Channel types,eeg
ICA components marked for exclusion,ICA000


### Well Done, we cleaned the data and can now go on to the even more fun part!